# BERT

## 1. Importing the libraries

In [1]:
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


## 1. Loading the dataset


In [5]:
import datasets

dataset = datasets.load_dataset("embedding-data/simple-wiki", split="train")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset

Dataset({
    features: ['set'],
    num_rows: 102225
})

Lets check a random sample from the dataset

In [7]:
dataset['set'][:5]

[['The greatest example has been in his present job ( then , Minister for Foreign Affairs ) , where he has perforce concentrated on Anglo-Irish relations and , in particular Northern Ireland ( .',
  'The greatest example has been in his present job ( then , Minister for Foreign Affairs ) , where he has perforce concentrated on Anglo-Irish relations and , in particular the North ( i.e. , Northern Ireland ) .'],
 ["President Hillery refused to speak to any opposition party politicians , but when Charles Haughey , who was Leader of the Opposition , had rang the President 's Office he threatened to end the career of the army officer answered and refused on Hillery 's explicit orders to put the call through to the President .",
  "His reputation rose further when opposition leaders under parliamentary privilege alleged that Taoiseach Charles Haughey , who in January 1982 had been Leader of the Opposition , had not merely rung the President 's Office but threatened to end the career of the a

## 3. Preprocessing

Remove the line breaks from all the sentence

In [8]:
sentences = [sentence for sublist in dataset['set'] for sentence in sublist]
len(sentences)

204450

In [9]:
sentences = [x.lower() for x in sentences]
sentences = [re.sub("[.,!?\\-]", '', x) for x in sentences]

In [10]:
# combine everything into one to make vocabs
word_list = list(set(" ".join(sentences).split()))
word2id = {w: i + 4 for i, w in enumerate(word_list)}
word2id.update({'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3})
id2word = {i: w for w, i in word2id.items()}

In [11]:
vocab_size = len(word2id)
vocab_size

120590

In [11]:
#list of all tokens for whole text
token_list = [[word2id[word] for word in sentence.split()] for sentence in sentences]

### Making vocabs

Before making the vocabs, let's remove all question marks and perios, etc, then turn everything to lowercase, and then simply split the text.

In [12]:
#take a look at token_list
token_list

[[108568,
  7600,
  115698,
  93202,
  53800,
  247,
  74234,
  17047,
  110513,
  75538,
  31157,
  7456,
  90522,
  15228,
  95624,
  44205,
  36273,
  86714,
  93202,
  98711,
  13083,
  33691,
  41341,
  64273,
  102586,
  247,
  35599,
  64871,
  86757,
  75538],
 [108568,
  7600,
  115698,
  93202,
  53800,
  247,
  74234,
  17047,
  110513,
  75538,
  31157,
  7456,
  90522,
  15228,
  95624,
  44205,
  36273,
  86714,
  93202,
  98711,
  13083,
  33691,
  41341,
  64273,
  102586,
  247,
  35599,
  108568,
  87742,
  75538,
  75070,
  64871,
  86757,
  44205],
 [35725,
  56658,
  14401,
  73896,
  60527,
  73896,
  96812,
  83367,
  36023,
  78245,
  93054,
  1475,
  119337,
  63600,
  16193,
  52087,
  46546,
  72911,
  108568,
  83367,
  25062,
  40511,
  108568,
  35725,
  53301,
  79270,
  86714,
  108416,
  73896,
  60162,
  108568,
  79760,
  72911,
  108568,
  36677,
  101148,
  58985,
  102586,
  14401,
  33691,
  56658,
  53301,
  98088,
  78504,
  73896,
  74520,
  10

In [13]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

the
greatest
example
has
been
in
his
present
job
(
then
minister
for
foreign
affairs
)
where
he
has
perforce
concentrated
on
angloirish
relations
and
in
particular
northern
ireland
(


## 2. Data loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given “The cat is walking. The dog is barking”, we add [CLS] and [SEP] >> “[CLS] the cat is walking [SEP] the dog is barking”.

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred`

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`.

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [14]:
batch_size = 6
max_mask   = 5  # max masked tokens when 15% exceed, it will only be max_pred
max_len    = 1000 # maximum of length to be padded;

In [18]:
def make_batch():
    batch = []
    positive = negative = 0  #count of batch size;  we want to have half batch that are positive pairs (i.e., next sentence pairs)
    while positive != batch_size/2 or negative != batch_size/2:

        #randomly choose two sentence so we can put [SEP]
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences))
        #retrieve the two sentences
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - append CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

        #2. segment embedding - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        #3. mask language modeling
        #masked 15%, but should be at least 1 but does not exceed max_mask
        n_pred =  min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get the pos that excludes CLS and SEP and shuffle them
        cand_maked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]'] and token != word2id['[SEP]']]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and change the input_ids to [MASK]
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)  #remember the position
            masked_tokens.append(input_ids[pos]) #remember the tokens
            #80% replace with a [MASK], but 10% will replace with a random token
            if random() < 0.1:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word2id[id2word[index]] # replace
            elif random() < 0.9:  # 80%
                input_ids[pos] = word2id['[MASK]'] # make mask
            else:  #10% do nothing
                pass

        # pad the input_ids and segment ids until the max len
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # pad the masked_tokens and masked_pos to make sure the lenth is max_mask
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #check if first sentence is really comes before the second sentence
        #also make sure positive is exactly half the batch size
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1

    return batch

In [19]:
batch = make_batch()

In [20]:
#len of batch
len(batch)

6

In [21]:
#we can deconstruct using map and zip
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape

(torch.Size([6, 1000]),
 torch.Size([6, 1000]),
 torch.Size([6, 5]),
 torch.Size([6, 5]),
 torch.Size([6]))

## 3. Model

Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

## 3.1 Embedding

Here we simply generate the positional embedding, and sum the token embedding, positional embedding, and segment embedding together.

<img src = "figures/BERT_embed.png" width=500>

In [49]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

## 3.2 Attention mask

In [50]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Testing the attention mask

In [24]:
print(get_attn_pad_mask(input_ids, input_ids).shape)

torch.Size([6, 1000, 1000])


## 3.3 Encoder

The encoder has two main components:

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [51]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_inputs_device = enc_inputs.device  # Get the device of enc_inputs
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask.to(enc_inputs_device))
        # Move enc_self_attn_mask to the device of enc_inputs
        enc_outputs = self.pos_ffn(enc_outputs)
        return enc_outputs, attn


Let's define the scaled dot attention, to be used inside the multihead attention

In [52]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

Let's define the parameters first

In [53]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

Here is the Multiheadattention.

In [54]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn # output: [batch_size x len_q x d_model]


Here is the PoswiseFeedForwardNet.

In [55]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))


## 3.4 Putting them together

In [56]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]

        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp

## 4. Training

In [57]:
num_epoch = 100
model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

# input_ids = input_ids.to(device)
# segment_ids = segment_ids.to(device)
# masked_pos = masked_pos.to(device)

for epoch in range(num_epoch):
    print("Current Epoch: " + str(epoch))
    optimizer.zero_grad()

    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification

    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 100 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Current Epoch: 0
Epoch: 00 loss = 130.115585
Current Epoch: 1
Current Epoch: 2
Current Epoch: 3
Current Epoch: 4
Current Epoch: 5
Current Epoch: 6
Current Epoch: 7
Current Epoch: 8
Current Epoch: 9
Current Epoch: 10
Current Epoch: 11
Current Epoch: 12
Current Epoch: 13
Current Epoch: 14
Current Epoch: 15
Current Epoch: 16
Current Epoch: 17
Current Epoch: 18
Current Epoch: 19
Current Epoch: 20
Current Epoch: 21
Current Epoch: 22
Current Epoch: 23
Current Epoch: 24
Current Epoch: 25
Current Epoch: 26
Current Epoch: 27
Current Epoch: 28
Current Epoch: 29
Current Epoch: 30
Current Epoch: 31
Current Epoch: 32
Current Epoch: 33
Current Epoch: 34
Current Epoch: 35
Current Epoch: 36
Current Epoch: 37
Current Epoch: 38
Current Epoch: 39
Current Epoch: 40
Current Epoch: 41
Current Epoch: 42
Current Epoch: 43
Current Epoch: 44
Current Epoch: 45
Current Epoch: 46
Current Epoch: 47
Current Epoch: 48
Current Epoch: 49
Current Epoch: 50
Current Epoch: 51
Current Epoch: 52
Current Epoch: 53
Current Ep

## 5. Inference

Since our dataset is very small, it won't work very well, but just for the sake of demonstration.

In [58]:
# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[2]))
print([id2word[w.item()] for w in input_ids[0] if id2word[w.item()] != '[PAD]'])

logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)
#logits_lm:  (1, max_mask, vocab_size) ==> (1, 5, 34)
#logits_nsp: (1, yes/no) ==> (1, 2)

#predict masked tokens
#max the probability along the vocab dim (2), [1] is the indices of the max, and [0] is the first value
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
#note that zero is padding we add to the masked_tokens
print('masked tokens (words) : ',[id2word[pos.item()] for pos in masked_tokens[0]])
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0]])
print('masked tokens (words) : ',[id2word[pos.item()] for pos in logits_lm])
print('predict masked tokens list : ', [pos for pos in logits_lm])

#predict nsp
logits_nsp = logits_nsp.data.max(1)[1][0].data.numpy()
print(logits_nsp)
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_nsp else False)

['[CLS]', 'tebay', 'is', 'a', '[MASK]', '[MASK]', 'civil', 'parish', 'near', 'the', 'm6', 'motorway', 'in', 'cumbria', 'england', '[SEP]', 'mã', 'ziã', 'ressuroise', 'is', 'a', 'commune', 'in', '[MASK]', 'aisne', '[MASK]', 'in', 'picardy', 'in', 'northern', 'france', '[SEP]']
masked tokens (words) :  ['village', 'department', 'the', 'a', 'and']
masked tokens list :  [37184, 117095, 108568, 100832, 102586]
masked tokens (words) :  ['and', 'and', 'and', 'united', 'and']
predict masked tokens list :  [102586, 102586, 102586, 7994, 102586]
1
isNext :  False
predict isNext :  True


Trying a bigger dataset should be able to see the difference.

# 6. Saving the model for future use

In [65]:
# Save Model
model_params = list(model.parameters())
model_state_dict = model.state_dict()

# Save model parameters and state dictionary
torch.save([model_params, model_state_dict], 'bert.pt')